In [6]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("nyc") \
    .getOrCreate()
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Reading Data

In [7]:
tickets = spark.read.csv('s3a://upgrad-spark-data/Parking_Violations_Issued_-_Fiscal_Year_2017.csv', header = True, inferSchema = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Data Exploration

In [8]:
tickets.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

In [9]:
from pyspark.sql.functions import *
pattern = 'MM/dd/yyyy'
tickets = tickets.withColumn('Issue Date', unix_timestamp(tickets['Issue Date'], pattern).cast('timestamp'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
tickets.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation C

In [11]:
tickets.select('Issue Date').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|         Issue Date|
+-------------------+
|2016-07-10 00:00:00|
|2016-07-08 00:00:00|
|2016-08-23 00:00:00|
|2017-06-14 00:00:00|
|2016-11-21 00:00:00|
|2017-06-13 00:00:00|
|2016-08-03 00:00:00|
|2016-12-21 00:00:00|
|2016-11-21 00:00:00|
|2016-10-05 00:00:00|
|2017-01-11 00:00:00|
|2016-09-27 00:00:00|
|2016-10-27 00:00:00|
|2016-09-30 00:00:00|
|2017-02-04 00:00:00|
|2016-07-07 00:00:00|
|2016-09-24 00:00:00|
|2017-01-26 00:00:00|
|2017-04-30 00:00:00|
|2017-02-03 00:00:00|
+-------------------+
only showing top 20 rows

In [12]:
tickets_2017 = tickets.filter(year(tickets['Issue Date'])==2017)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
tickets_2017.select('Issue Date').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|         Issue Date|
+-------------------+
|2017-06-14 00:00:00|
|2017-06-13 00:00:00|
|2017-01-11 00:00:00|
|2017-02-04 00:00:00|
|2017-01-26 00:00:00|
|2017-04-30 00:00:00|
|2017-02-03 00:00:00|
|2017-05-27 00:00:00|
|2017-05-31 00:00:00|
|2017-05-26 00:00:00|
|2017-05-19 00:00:00|
|2017-06-09 00:00:00|
|2017-01-20 00:00:00|
|2017-04-13 00:00:00|
|2017-01-05 00:00:00|
|2017-02-18 00:00:00|
|2017-06-14 00:00:00|
|2017-01-29 00:00:00|
|2017-01-25 00:00:00|
|2017-06-12 00:00:00|
+-------------------+
only showing top 20 rows

In [14]:
# 1. Find the total number of tickets issued in the year 2017
tickets_2017.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5431918

In [15]:
# For using SQL, you need to create a temporary view
tickets_2017.createOrReplaceTempView("data_2017")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# 2. Find out the number of unique states from where the cars 
# that got parking tickets came from. (Hint: Use 'Registration State')
spark.sql(""" SELECT count(DISTINCT `Registration State`) AS count \
                FROM data_2017 \
                """).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+
|count|
+-----+
|   65|
+-----+

In [17]:
# Arranging the dataframe based on number of entries
spark.sql("""
SELECT `Registration State`, count(*) As count \
FROM data_2017 \
GROUP BY `Registration State` \
ORDER BY count DESC
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------+
|Registration State|  count|
+------------------+-------+
|                NY|4273951|
|                NJ| 475825|
|                PA| 140286|
|                CT|  70403|
|                FL|  69468|
|                IN|  45525|
|                MA|  38941|
|                VA|  34367|
|                MD|  30213|
|                NC|  27152|
|                TX|  18827|
|                IL|  18666|
|                GA|  17537|
|                99|  16055|
|                AZ|  12379|
|                OH|  12281|
|                CA|  12153|
|                ME|  10806|
|                SC|  10395|
|                MN|  10083|
+------------------+-------+
only showing top 20 rows

In [18]:
#Replacing '99' with 'NY' in the dataframe
from pyspark.sql import functions as F
tickets_2017_new = tickets_2017.withColumn('Registration State', F.when(tickets_2017['Registration State']=='99', 'NY')\
                                           .otherwise(tickets_2017['Registration State']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# The temporary view needs to be recreated as values have been updated in tickets_2017_new
tickets_2017_new.createOrReplaceTempView("data_2017_new")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
spark.sql(
""" SELECT `Registration State`, count(*) AS count \
    FROM data_2017_new \
    GROUP BY `Registration State` \
    ORDER BY count DESC
    """).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------+
|Registration State|  count|
+------------------+-------+
|                NY|4290006|
|                NJ| 475825|
|                PA| 140286|
|                CT|  70403|
|                FL|  69468|
|                IN|  45525|
|                MA|  38941|
|                VA|  34367|
|                MD|  30213|
|                NC|  27152|
|                TX|  18827|
|                IL|  18666|
|                GA|  17537|
|                AZ|  12379|
|                OH|  12281|
|                CA|  12153|
|                ME|  10806|
|                SC|  10395|
|                MN|  10083|
|                OK|   9088|
+------------------+-------+
only showing top 20 rows

In [22]:
# Find out the number of unique states from where the cars 
# that got parking tickets came from. (Hint: Use 'Registration State')
spark.sql(""" SELECT count(DISTINCT `Registration State`) AS count \
                FROM data_2017_new \
                """).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+
|count|
+-----+
|   64|
+-----+

## Aggregation Tasks

In [23]:
spark.sql('''

SELECT `Violation Code`, count(*) AS count \
FROM data_2017_new \
GROUP BY `Violation Code` \
ORDER BY count DESC \
LIMIT 5
''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
|            14|476664|
|            20|319646|
+--------------+------+

In [24]:
spark.sql('''

SELECT `Vehicle Body Type`, count(*) AS count \
FROM data_2017_new \
GROUP BY `Vehicle Body Type` \
ORDER BY count DESC \
LIMIT 5
''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------+
|Vehicle Body Type|  count|
+-----------------+-------+
|             SUBN|1883954|
|             4DSD|1547312|
|              VAN| 724029|
|             DELV| 358984|
|              SDN| 194197|
+-----------------+-------+

In [25]:
spark.sql('''

SELECT `Vehicle Make`, count(*) AS count \
FROM data_2017_new \
GROUP BY `Vehicle Make` \
ORDER BY count DESC \
LIMIT 5
''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+
|Vehicle Make| count|
+------------+------+
|        FORD|636844|
|       TOYOT|605291|
|       HONDA|538884|
|       NISSA|462017|
|       CHEVR|356032|
+------------+------+

In [26]:
tickets_seasonality = spark.sql('''select `Violation Code`, `Issue Date`, case
                                  when month(to_date(`Issue Date`, 'MM/dd/yyyy')) between 03 and 05
                                  then 'spring'
                                  when month(to_date(`Issue Date`, 'MM/dd/yyyy')) between 06 and 08
                                  then 'summer'
                                  when month(to_date(`Issue Date`, 'MM/dd/yyyy')) between 09 and 11
                                  then 'autumn'
                                  when month(to_date(`Issue Date`, 'MM/dd/yyyy')) in (1,2,12)
                                  then 'winter'
                                  else 'unknown'
                                  end as season
                                  from data_2017_new''')
tickets_seasonality.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------------+------+
|Violation Code|         Issue Date|season|
+--------------+-------------------+------+
|            47|2017-06-14 00:00:00|summer|
|             7|2017-06-13 00:00:00|summer|
|            78|2017-01-11 00:00:00|winter|
|            40|2017-02-04 00:00:00|winter|
|            64|2017-01-26 00:00:00|winter|
|            20|2017-04-30 00:00:00|spring|
|            36|2017-02-03 00:00:00|winter|
|            38|2017-05-27 00:00:00|spring|
|            14|2017-05-31 00:00:00|spring|
|            75|2017-05-26 00:00:00|spring|
|            10|2017-05-19 00:00:00|spring|
|            69|2017-06-09 00:00:00|summer|
|            21|2017-01-20 00:00:00|winter|
|            38|2017-04-13 00:00:00|spring|
|            48|2017-01-05 00:00:00|winter|
|            21|2017-02-18 00:00:00|winter|
|            21|2017-06-14 00:00:00|summer|
|            68|2017-01-29 00:00:00|winter|
|            51|2017-01-25 00:00:00|winter|
|             9|2017-06-12 00:00

In [27]:
# For using SQL, you need to create a temporary view
tickets_seasonality.createOrReplaceTempView('seasonal_data')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Grouping tickets by season
spark.sql(
"""
SELECT season, count(*) as No_of_tickets
FROM seasonal_data
GROUP BY season
ORDER BY No_of_tickets DESC
"""
).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+
|season|No_of_tickets|
+------+-------------+
|spring|      2873383|
|winter|      1704690|
|summer|       852866|
|autumn|          979|
+------+-------------+

In [29]:
#Spring
spark.sql(
"""
SELECT season, `Violation Code`, count(*) as No_of_tickets
FROM seasonal_data
WHERE season = 'spring'
GROUP BY season, `Violation Code`
ORDER BY No_of_tickets DESC
LIMIT 3
"""
).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+-------------+
|season|Violation Code|No_of_tickets|
+------+--------------+-------------+
|spring|            21|       402424|
|spring|            36|       344834|
|spring|            38|       271167|
+------+--------------+-------------+

In [30]:
#Winter
spark.sql(
"""
SELECT season, `Violation Code`, count(*) as No_of_tickets
FROM seasonal_data
WHERE season = 'winter'
GROUP BY season, `Violation Code`
ORDER BY No_of_tickets DESC
LIMIT 3
"""
).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+-------------+
|season|Violation Code|No_of_tickets|
+------+--------------+-------------+
|winter|            21|       238183|
|winter|            36|       221268|
|winter|            38|       187386|
+------+--------------+-------------+

In [31]:
#Summer
spark.sql(
"""
SELECT season, `Violation Code`, count(*) as No_of_tickets
FROM seasonal_data
WHERE season = 'summer'
GROUP BY season, `Violation Code`
ORDER BY No_of_tickets DESC
LIMIT 3
"""
).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+-------------+
|season|Violation Code|No_of_tickets|
+------+--------------+-------------+
|summer|            21|       127352|
|summer|            36|        96663|
|summer|            38|        83518|
+------+--------------+-------------+

In [32]:
#Autumn
spark.sql(
"""
SELECT season, `Violation Code`, count(*) as No_of_tickets
FROM seasonal_data
WHERE season = 'autumn'
GROUP BY season, `Violation Code`
ORDER BY No_of_tickets DESC
LIMIT 3
"""
).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+-------------+
|season|Violation Code|No_of_tickets|
+------+--------------+-------------+
|autumn|            46|          231|
|autumn|            21|          128|
|autumn|            40|          116|
+------+--------------+-------------+

In [34]:
## Total occurrences of the 3 most common violation codes
spark.sql("select `Violation Code`, count(*) as `no_of_tickets`\
                    from data_2017_new\
                    group by `Violation Code`\
                    order by `no_of_tickets` desc\
                    limit 3").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------+
|Violation Code|no_of_tickets|
+--------------+-------------+
|            21|       768087|
|            36|       662765|
|            38|       542079|
+--------------+-------------+

In [35]:
#Calculating Fine
from pyspark.sql.functions import *
tickets_2017_new = tickets_2017_new.withColumn('Fine Amount', when(tickets_2017_new["Violation Code"]==21, 65) \
                                               .otherwise(when(tickets_2017_new["Violation Code"]==36,50) \
                                                          .otherwise(when(tickets_2017_new["Violation Code"]==38,50) \
                                                                     .otherwise(0))))
tickets_2017_new.createOrReplaceTempView("new_data_2017")
spark.sql("""
SELECT `Violation Code`, SUM(`Fine Amount`) AS Fine_Amount_Total
FROM new_data_2017 \
GROUP BY `Violation Code` \
ORDER BY Fine_Amount_Total DESC
LIMIT 3
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----------------+
|Violation Code|Fine_Amount_Total|
+--------------+-----------------+
|            21|         49925655|
|            36|         33138250|
|            38|         27103950|
+--------------+-----------------+

In [36]:
spark.sql("""
SELECT `Registration State`, SUM(`Fine Amount`) AS Fine_Amount_Total
FROM new_data_2017 \
GROUP BY `Registration State` \
ORDER BY Fine_Amount_Total DESC
LIMIT 3
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+
|Registration State|Fine_Amount_Total|
+------------------+-----------------+
|                NY|         87901590|
|                NJ|          7368020|
|                PA|          3413995|
+------------------+-----------------+